<a href="https://colab.research.google.com/github/ansonkwokth/TableTennisPrediction/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/ansonkwokth/TableTennisPrediction.git
%cd TableTennisPrediction

fatal: destination path 'TableTennisPrediction' already exists and is not an empty directory.
/content/TableTennisPrediction


In [179]:
import pandas as pd
from tabulate import tabulate
from utils import data_loader as dl

import numpy as np
from model.Elo import Elo
from model.ModifiedElo import ModifiedElo
from model.ensemble import BaggingRatingSystem

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import copy

import warnings
warnings.filterwarnings('ignore')

In [180]:
GAME = 'TTStar'
# GAME = 'TTCup'
# GAME = 'SetkaCup'
# GAME = 'SetkaCupWomen'
# GAME = 'LigaPro'


In [181]:
match GAME:
    case 'TTStar':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'TTCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCupWomen':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'LigaPro':
        years = [2022, 2023, 2024]
    case _:
        raise ValueError("Invalid game selected.")


text_data_game = dl.load_game_data(GAME, years, '../')
text_data = {
    year: text_data_game[year] for year in years
}
df = dl.create_game_dfs(GAME, years, text_data)

Loading ..//TTStar2020.txt
Loading ..//TTStar2021.txt
Loading ..//TTStar2022.txt
Loading ..//TTStar2023.txt
Loading ..//TTStar2024.txt


In [182]:
# Generate ID indices for each pair of rows in the DataFrame
idx_lt = [i for i in range(len(df) // 2) for _ in range(2)]
df['ID'] = idx_lt  # Assign to the 'ID' column

# Reset the DataFrame index to ensure it's sequential
df.reset_index(drop=True, inplace=True)

# Get unique players and store them in player_lt
player_lt = df['Player'].unique()



In [183]:
def format_to_array(df: pd.DataFrame) -> np.ndarray:

    # info_col = ['ID', 'Round', 'Datetime', 'Game', 'Date', 'Time']
    info_col = ['Round', 'Datetime', 'Game', 'Date', 'Time']
    col = [item for item in df.columns if item not in info_col]

    df[[c for c in col if "Set" in c]] = df[[c for c in col if "Set" in c]].astype(float)
    X = df[col].values.reshape(-1, 2, len(col))
    return X

In [184]:
X_all = format_to_array(df)

In [185]:
modelMElo = ModifiedElo()
modelMElo.fit(X_all)


Training model: 100%|██████████| 17257/17257 [00:00<00:00, 18400.71it/s]


# Print

In [186]:
def find_player(name):
    p_lt = []
    for k, v in modelMElo.params.items():
        if name in k:
            print(k)
            p_lt.append(k)
    return p_lt

In [187]:
def find_OU(player1, player2):
    set1_dt = modelMElo.predict_set_config(player1, player2)[-1]
    set2_dt = modelMElo.predict_set_config(player2, player1)[-1]

    set_sum_dt = {}
    for k1, v1 in set1_dt.items():
        v2 = set2_dt[k1]
        set_sum_dt[k1] = v1 + v2



    p_O = 0
    p_U = 0
    for k, v in set_sum_dt.items():
        if k > line:
            p_O += v
        else:
            p_U += v
    return p_O, p_U

In [188]:
df_lt = []

In [243]:
p1_lt = find_player("Gio")
print("-"*30)
p2_lt = find_player("Dev")

Giovannetti T.
------------------------------
Devos L.


In [244]:
player1 = p1_lt[0]
player2 = p2_lt[0]
player1, player2

('Giovannetti T.', 'Devos L.')

In [245]:
time = "16:30"

date = "Feb 5"


line = 18.5


found_p1, found_p2, p_game = modelMElo.predict_game(player1, player2)
found_p1, found_p2, p_set = modelMElo.predict_set(player1, player2)
p_O, p_U = find_OU(player1, player2)
winner = player1 if p_game > 0.5 else player2
p_set_win = p_set if winner == player1 else 1 - p_set
p_game_win = p_game if winner == player1 else 1 - p_game
p_OU = p_O if p_O > p_U else p_U
OU = "O" if p_O > p_U else "U"

In [246]:
if (found_p1 and found_p2):
    df_row = pd.DataFrame({
                         "Time": [time],
                         "Player1": [player1], "Player2": [player2], "Winner": [winner],
                         "Set": [f"{p_set_win*100:.1f}%"], "Game": [f"{p_game_win*100:.1f}%"]})
                        #  "O/U": [f"{OU} {line}: {p_OU*100:.1f}%"]})

    print(tabulate(df_row, headers = 'keys', tablefmt = 'psql', showindex=False))


+--------+----------------+-----------+----------+-------+--------+
| Time   | Player1        | Player2   | Winner   | Set   | Game   |
|--------+----------------+-----------+----------+-------+--------|
| 16:30  | Giovannetti T. | Devos L.  | Devos L. | 75.7% | 90.3%  |
+--------+----------------+-----------+----------+-------+--------+


In [247]:
p_U, p_O

(0.5567188091250238, 0.4432811908749762)

In [248]:
df_lt.append(df_row)

In [249]:
df_print = pd.concat(df_lt)
df_print.index = range(len(df_lt))
print(tabulate(df_print, headers = 'keys', tablefmt = 'psql', showindex=False))
# print(tabulate(df_print, headers = 'keys', tablefmt = 'psql'))

+--------+----------------+----------------+----------------+-------+--------+
| Time   | Player1        | Player2        | Winner         | Set   | Game   |
|--------+----------------+----------------+----------------+-------+--------|
| 09:00  | Gainza A.      | Nunez Tapia M. | Nunez Tapia M. | 72.0% | 86.2%  |
| 10:00  | Muhlbach C.    | Bhanja R.      | Bhanja R.      | 51.9% | 53.5%  |
| 11:30  | Muhlbach C.    | Devos L.       | Devos L.       | 63.3% | 73.8%  |
| 12:00  | Giovannetti T. | Nunez Tapia M. | Nunez Tapia M. | 56.2% | 61.4%  |
| 13:00  | Giovannetti T. | Muhlbach C.    | Muhlbach C.    | 63.8% | 74.6%  |
| 14:30  | Giovannetti T. | Bhanja R.      | Bhanja R.      | 65.8% | 77.7%  |
| 16:00  | Bhanja R.      | Nunez Tapia M. | Bhanja R.      | 59.9% | 68.1%  |
| 16:30  | Giovannetti T. | Devos L.       | Devos L.       | 75.7% | 90.3%  |
+--------+----------------+----------------+----------------+-------+--------+
